# Numerical Methods for Free-Boundary Parabolic Partial Differential Equations

In this doucment, we describe our approach to solving the free-boundary problem $$\begin{cases} \frac{\partial H}{\partial t} + \alpha S \frac{\partial H}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 H}{\partial S^2} = 0 & x \in [X_b(t, S), X_s(t, S)], \\ \frac{\partial H}{\partial x} + \alpha \gamma Se^{r(T - t)} H = 0 & x \leq X_b(t, S) \\ \frac{\partial H}{\partial x} + b \gamma Se^{r(T - t)}H = 0 & x \geq X_s(t, S), \\ H(T, S, x) = e^{-\gamma Z(S, x)},\end{cases} \tag{$\star$}$$

Where $\alpha, \sigma, \gamma, a, b$ are parameters and $Z(S, X)$ is given by $$Z(S, x) = xS(a I_{\{x > 0\}} + b I_{\{x \geq 0\}} \tag{$\star \star$}).$$

The problem ($\star$) is an example of an **double obstacle problem** (two moving boundaries). The suitable numerical method is the finite element method apporach together with characteristics time discretization and projected relaxation for handling the free boundaries. 

## Approach 1 Description (Legacy FEM Method)

1. **Problem Transformation and Localization**
- Set $\tau = T - t$ (backward time) so that in the PDE becomes: $$\begin{cases} \alpha S \frac{\partial H}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 H}{\partial S^2} = \frac{\partial H}{\partial \tau} & x \in [X_b(\tau, S), X_s(\tau, S)], \\  \frac{\partial H}{\partial x} + \alpha \gamma Se^{r \tau} H = 0 & x \leq X_b(\tau, S) \\ \frac{\partial H}{\partial x} + b \gamma Se^{r \tau} H = 0 & x \geq X_s(\tau, S) \\ H(T, S, x) = e^{-\gamma Z(S, x)}\end{cases} \tag{$\star \tau$}$$

- We truncate the domain of $S \in [S_{\min}, S_{\max}]$ and $x \in [x_\min, x_\max]$. At $S_\min$, we assume the stock price is near zero, so that the position value $Z \approx 0$ and $H \to 1$ giving the Dirichlet boundary condition $H(\tau, S_{\min}, x) = 1$. At $S_\max$, we assume that there is little change in the value function for very large stock prices, so we impose the artificial Neumann boundary condition $\frac{\partial H}{\partial S}(\tau, S_{\max}, x) = 0.$ For $x = x_\min$ and $x = x_\max$ we impose the Neumann boundary conditions $$\frac{\partial H}{\partial x}(\tau, S, x_{\min}) = - a \gamma Se^{r \tau} H(\tau, S, x_{\min}), \qquad \frac{\partial H}{\partial x}(\tau, S, x_{\max}) = - b \gamma Se^{r \tau} H(\tau, S, x_{\max}).$$

2. **Variational Inequality Formulation** Here, we reformulate the given free-boundary problem as a parabolic double-obstacle variational inequality (which it is).
- Let $V = H^1([x_{\min}, x_{\max}])$ be the **Sobolev space** of functions $v: [x_{\min}, x_{\max}] \to \mathbb{R}$ such that $v$ and **its weak derivative** $v'$ are square integrable: $$\int_{x_{\min}}^{x_{\max}} |v(x)|^2 \ dx < \infty, \qquad \int_{\min}^{\max} |v'(x)|^2 \ dx < \infty.$$ By weak derivative, we mean that $v'$ satisfies the equality $$\int_{x_{\min}}^{x_{\max}} v(x) \varphi(x) \ dx = -\int_{x_{\min}}^{x_{\max}}v'(x) \varphi(x) \ dx \tag{Weak Derivative Definition}$$ for all test functions $\varphi \in C_c^\infty(x_{\min}, x_{\max})$ which are infinitely-differentiable functions with compact support (compact support means they are zero outside some bounded set, a/k/a bump function). The Sobolev space $H^1$ is endowed with a norm: $$||u||_{H^1} = \left(\int_{x_{\min}}^{x_{\max}} |v(x)|^2 \ dx + \int_{x_{\min}}^{x_{\max}} |v'(x)|^2 \ dx\right)^{1/2}.$$
- Let $$\mathcal{K}(\tau, S) = \{v \in V \mid \underline{\psi}(\tau, S, x) \leq v(x) \leq \overline{\psi}(\tau, S, x) \ \forall x\}$$ where $\underline{\psi}$ and $\overline{\psi}$ are hte lwoer and upper obstacle functions defined by integrating the boundary ODEs given in bullet point 1 above over the buy and sell regions, respectively. So, $\underline{\psi}(\tau, S, x) = H(\tau, S, X_b) \exp(-a \gamma Se^{r \tau}(x - X_b))$, $\overline{\psi}(\tau, S, x) = H(\tau, S, X_s) \exp (-b, \gamma Se^{r \tau}(x - X_s))$. 
- The **"weak form"** of the problem is: Find $H(\tau, S, \cdot) \in \mathcal{K}(\tau, S)$ such that for almost every $S$, $$\left(\frac{\partial H}{\partial \tau}, v - H\right) + a(H, v - H) \geq 0, \qquad \forall v \in \mathcal{K}(\tau, S),$$ where $$a(u, v) = \int_{x_{\min}}^{x_{\max}} \left(-\alpha S \frac{\partial u}{\partial S} v - \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 u}{\partial S^2} v\right) \ dx$$ is a bilinear form.

3. **Time Discretization** (Implicit Euler Scheme)
- Very simple, divide $[0, T]$ into $N_t$ equal steps with $\Delta \tau = T/N_t$, $\tau^{n} := n \Delta \tau$ (**This is not the algebraic exponent - treat the exponent here as the same as a subscript denoting a term in a sequence**).
- We use the **implicit Euler schem with characteristics** for the material derivative in $S$: $$\frac{H^{n + 1}(S, x) - H^n(\hat S, x)}{\Delta t} = \alpha S \frac{\partial H}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 H}{\partial S^2}$$ where $\hat S = S \exp ((\alpha - \frac{1}{2} \sigma^2) \Delta \tau)$.

4. **Spatial Discretization**
- In $x$: Use piecewise linear finite elements on a uniform mesh with nodes $x_j = x_{\min} + jh_x$, $j = 0, \dots, N_x$, for $h_x = (x_{\max} - x_{\min})/N_x$.
- In $S$: Use a nonuniform log-grif $S_i = S_\min \exp (ih_S), h_S = \frac{1}{N_S} \log(S_{\max}/S_{\min})$, and finite differences (upwind for $\alpha S \partial_S$, central for $\sigma^2 S^2 \partial_S^2$).
- The discrete spcae $V_h = \{v_h \in C^0([x_{\min}, x_{\max}]) \mid v_h \mid_{x_j, x_{j + 1}} \in \mathbb{P}^1\}$ (pw linear).
- With the mass matrix $M$ and stiffness matrix $A$ assembled per fixed $x$-node, forming a block-diagonal system coupled with $x$-discretization, we have that the discrete VI at time $\tau^{n + 1}$: Find $H_h^{n + 1} \in \mathcal{K}_h^{n + 1}$ such that $$\left(M + \frac{H_h^{n + 1} - H_h^n \circ \chi}{\Delta \tau}, v_h - H_h^{n + 1}\right)_h + a_h (H_h^{n + 1}, v_h - H_h^{n + 1}) \geq 0, \qquad \forall v_h \in \mathcal{K}_h^{n + 1}$$ where $\chi$ is the characteristics map, $(\cdot, \cdot)_h$ is the lumped mass inner product, and $\mathcal{K}_h^{n + 1}$ enforces obstacles at nodes. 

5. **Algorithm**
   - **Step 1** Initialize $H_h^{n + 1, 0} = H_h^n \circ \chi$,
   - **Step 2** For each $k = 0, 1, \dots,$ until convergence (see step 3), for each node $(i, j)$ we set $$\hat H_{i, j} = \frac{q_{i,j} - \sum_{l \neq j} A_{i,j,l} H_{i,l}^{n + 1, k + 1/2}}{A_{i,j,j}}$$ where $q = M(H_h^n \circ \chi)/\Delta \tau$, $A$ is the discrete operator matrix, and the half-update uses newest values for $l < j$, old for $l > j$. We then project, with the relaxation parameter $\omega \in (1, 2)$, $$H_{i,j}^{n + 1, k + 1} = \min\left(\max\left(\underline{\psi}_{i,j}^{n + 1}, H_{i,j}^{n + 1, k} + \omega (\hat H_{i,j} - H_{i,j}^{n + 1, k})\right), \overline{\psi}_{i,j}^{n + 1}\right)$$
   - **Step 3** Stop when $||H^{n + 1, k + 1} - H^{n + 1, k}||_\infty < \epsilon$.